# SparQL Data Retriever


Sparql wrapper (copy and pasted from the Individual Project's notebooks):

In [12]:

from SPARQLWrapper import SPARQLWrapper, JSON

# Actually, the "Ministero dei Beni Culturali" has something like 30
# different namespaces, it was just easier to copy and paste the full urls.
prefixString = """

"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("https://dati.cultura.gov.it/sparql") # Endpoint of "Ministero dei Beni Culturali"
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            array.append(app)
        return array

    except Exception as e :
        print("The operation failed", e)
    

Get categories of artworks (will be used programmatically later on):

In [13]:
categories = run_query("""
select distinct ?category where {
    ?category rdfs:subClassOf* <https://w3id.org/arco/ontology/arco/TangibleCulturalProperty> .

}
LIMIT 3000
""")

print(len(categories))
categories



17


[[('category',
   'https://w3id.org/arco/ontology/arco/TangibleCulturalProperty')],
 [('category', 'https://w3id.org/arco/ontology/arco/MovableCulturalProperty')],
 [('category',
   'https://w3id.org/arco/ontology/arco/ImmovableCulturalProperty')],
 [('category', 'https://w3id.org/arco/ontology/arco/ArchaeologicalProperty')],
 [('category',
   'https://w3id.org/arco/ontology/arco/ArchitecturalOrLandscapeHeritage')],
 [('category',
   'https://w3id.org/arco/ontology/arco/ScientificOrTechnologicalHeritage')],
 [('category', 'https://w3id.org/arco/ontology/arco/PhotographicHeritage')],
 [('category', 'https://w3id.org/arco/ontology/arco/NumismaticProperty')],
 [('category', 'https://w3id.org/arco/ontology/arco/NaturalHeritage')],
 [('category', 'https://w3id.org/arco/ontology/arco/MusicHeritage')],
 [('category',
   'https://w3id.org/arco/ontology/arco/HistoricOrArtisticProperty')],
 [('category', 'https://w3id.org/arco/ontology/arco/ZoologicalHeritage')],
 [('category', 'https://w3id.org

In [14]:


measureTypes = run_query("""
    select distinct ?measureType {
        ?artwork a <https://w3id.org/arco/ontology/arco/HistoricOrArtisticProperty> .
        ?artwork <https://w3id.org/arco/ontology/location/hasCulturalInstituteOrSite> ?culturalSite.
        
        ?culturalSite ?p ?site.
        ?site <http://dati.beniculturali.it/cis/siteAddress> ?address.
        ?address  <https://w3id.org/italia/onto/CLV/hasCity> <https://w3id.org/arco/resource/City/padova>.

        OPTIONAL {
            ?artwork <http://purl.org/dc/elements/1.1/format> ?geom.
            ?geom <https://w3id.org/arco/ontology/denotative-description/hasMeasurement> ?measureNode.
            ?measureNode <https://w3id.org/arco/ontology/denotative-description/hasMeasurementType> ?measureType.
        }

    }
    LIMIT 3000
    """)
print(len(measureTypes))
measureTypes

7


[[('measureType',
   'https://w3id.org/arco/ontology/denotative-description/Height')],
 [('measureType',
   'https://w3id.org/arco/ontology/denotative-description/Width')],
 [('measureType',
   'https://w3id.org/arco/ontology/denotative-description/Thickness')],
 [('measureType',
   'https://w3id.org/arco/ontology/denotative-description/Length')],
 [('measureType',
   'https://w3id.org/arco/ontology/denotative-description/Diameter')],
 [('measureType',
   'https://w3id.org/arco/ontology/denotative-description/Depth')],
 []]

In [15]:
things = {}
for [(_,category)] in categories:
    a = run_query("""
    select distinct ?artwork ?name ?desc ?desc2 ?desc3 ?material ?img ?img2 ?date ?tag ?conservation ?measures
    ?height ?width ?thickness ?length ?diameter ?depth where {
        ?artwork a <""" + category + """> .
        ?artwork <https://w3id.org/arco/ontology/location/hasCulturalInstituteOrSite> ?culturalSite.
        
        ?culturalSite ?p ?site.
        ?site <http://dati.beniculturali.it/cis/siteAddress> ?address.
        ?address  <https://w3id.org/italia/onto/CLV/hasCity> <https://w3id.org/arco/resource/City/padova>.
        

    }
    LIMIT 6000
    """)
    print(category, len(a))
    things[category] = a




Empty
https://w3id.org/arco/ontology/arco/TangibleCulturalProperty 0
https://w3id.org/arco/ontology/arco/MovableCulturalProperty 2939
Empty
https://w3id.org/arco/ontology/arco/ImmovableCulturalProperty 0
https://w3id.org/arco/ontology/arco/ArchaeologicalProperty 468
Empty
https://w3id.org/arco/ontology/arco/ArchitecturalOrLandscapeHeritage 0
https://w3id.org/arco/ontology/arco/ScientificOrTechnologicalHeritage 20
https://w3id.org/arco/ontology/arco/PhotographicHeritage 8
https://w3id.org/arco/ontology/arco/NumismaticProperty 4
https://w3id.org/arco/ontology/arco/NaturalHeritage 805
Empty
https://w3id.org/arco/ontology/arco/MusicHeritage 0
https://w3id.org/arco/ontology/arco/HistoricOrArtisticProperty 1436
Empty
https://w3id.org/arco/ontology/arco/ZoologicalHeritage 0
Empty
https://w3id.org/arco/ontology/arco/PetrologicHeritage 0
Empty
https://w3id.org/arco/ontology/arco/PalaeontologicalHeritage 0
Empty
https://w3id.org/arco/ontology/arco/MineralHeritage 0
Empty
https://w3id.org/arco/on

In [16]:
properties = {}

index = 0

avoidRepeat = []

for [(_,category)] in categories:
    for [(_,artwork)] in things[category]:
        if(artwork in avoidRepeat): continue
        avoidRepeat.append(artwork)
        a = run_query("""
        select distinct ?artwork ?name ?site ?desc ?desc2 ?desc3 ?material ?img ?img2 ?date ?tag ?conservation
        ?height ?width ?thickness ?length ?diameter ?depth where {

            BIND(<""" + artwork + """> AS ?artwork)

            ?artwork <https://w3id.org/arco/ontology/location/hasCulturalInstituteOrSite> ?culturalSite.
            
            ?culturalSite ?p ?site.
            ?site <http://dati.beniculturali.it/cis/siteAddress> ?address.
            ?address  <https://w3id.org/italia/onto/CLV/hasCity> <https://w3id.org/arco/resource/City/padova>.

            OPTIONAL { ?artwork <http://www.w3.org/2000/01/rdf-schema#label> ?name. }
            OPTIONAL { ?artwork <http://purl.org/dc/elements/1.1/description> ?desc. }
            OPTIONAL { ?artwork <https://w3id.org/arco/ontology/core/description> ?desc2. }
            OPTIONAL { ?artwork <http://purl.org/dc/elements/1.1/subject> ?desc3. }
            OPTIONAL { ?artwork <http://data.cochrane.org/ontologies/pico/materialAndTechnique> ?material. }
            OPTIONAL { ?artwork <http://data.cochrane.org/ontologies/pico/preview> ?img. }
            OPTIONAL { ?artwork <http://xmlns.com/foaf/0.1/depiction> ?img2. }
            OPTIONAL { ?artwork <http://purl.org/dc/elements/1.1/date> ?date. }
            OPTIONAL { ?artwork <http://purl.org/dc/elements/1.1/type> ?tag. }            

    
            OPTIONAL { 
                ?artwork <https://w3id.org/arco/ontology/denotative-description/hasConservationStatus> ?conservationNode.
                ?conservationNode <https://w3id.org/arco/ontology/core/specifications> ?conservation.
            }

            
            OPTIONAL {
                ?artwork <http://purl.org/dc/elements/1.1/format> ?geom.
                OPTIONAL {
                    ?geom <https://w3id.org/arco/ontology/denotative-description/hasMeasurement> ?measureNode1.
                    ?measureNode1 <https://w3id.org/arco/ontology/denotative-description/hasMeasurementType> <https://w3id.org/arco/ontology/denotative-description/Height>.
                    ?measureNode1 <https://w3id.org/arco/ontology/denotative-description/hasValue> ?mValue1.
                    ?mValue1 <https://w3id.org/italia/onto/MU/value> ?height.
                }

                OPTIONAL {
                    ?geom <https://w3id.org/arco/ontology/denotative-description/hasMeasurement> ?measureNode2.
                    ?measureNode2 <https://w3id.org/arco/ontology/denotative-description/hasMeasurementType> <https://w3id.org/arco/ontology/denotative-description/Width>.
                    ?measureNode2 <https://w3id.org/arco/ontology/denotative-description/hasValue> ?mValue2.
                    ?mValue2 <https://w3id.org/italia/onto/MU/value> ?width.
                }


                OPTIONAL {
                    ?geom <https://w3id.org/arco/ontology/denotative-description/hasMeasurement> ?measureNode3.
                    ?measureNode3 <https://w3id.org/arco/ontology/denotative-description/hasMeasurementType> <https://w3id.org/arco/ontology/denotative-description/Thickness>.
                    ?measureNode3 <https://w3id.org/arco/ontology/denotative-description/hasValue> ?mValue3.
                    ?mValue3 <https://w3id.org/italia/onto/MU/value> ?thickness.
                }


                OPTIONAL {
                    ?geom <https://w3id.org/arco/ontology/denotative-description/hasMeasurement> ?measureNode4.
                    ?measureNode4 <https://w3id.org/arco/ontology/denotative-description/hasMeasurementType> <https://w3id.org/arco/ontology/denotative-description/Length>.
                    ?measureNode4 <https://w3id.org/arco/ontology/denotative-description/hasValue> ?mValue4.
                    ?mValue4 <https://w3id.org/italia/onto/MU/value> ?length.
                }


                OPTIONAL {
                    ?geom <https://w3id.org/arco/ontology/denotative-description/hasMeasurement> ?measureNode5.
                    ?measureNode5 <https://w3id.org/arco/ontology/denotative-description/hasMeasurementType> <https://w3id.org/arco/ontology/denotative-description/Diameter>.
                    ?measureNode5 <https://w3id.org/arco/ontology/denotative-description/hasValue> ?mValue5.
                    ?mValue5 <https://w3id.org/italia/onto/MU/value> ?diameter.
                }


                OPTIONAL {
                    ?geom <https://w3id.org/arco/ontology/denotative-description/hasMeasurement> ?measureNode6.
                    ?measureNode6 <https://w3id.org/arco/ontology/denotative-description/hasMeasurementType> <https://w3id.org/arco/ontology/denotative-description/Depth>.
                    ?measureNode6 <https://w3id.org/arco/ontology/denotative-description/hasValue> ?mValue6.
                    ?mValue6 <https://w3id.org/italia/onto/MU/value> ?depth.
                }
            }

                                
            

        }
        LIMIT 1
        """)
        index = index + 1
        print(index, artwork)
    
        properties[artwork] = a



1 https://w3id.org/arco/resource/ArchaeologicalProperty/0500590655
2 https://w3id.org/arco/resource/ArchaeologicalProperty/0500590658
3 https://w3id.org/arco/resource/ArchaeologicalProperty/0500590661
4 https://w3id.org/arco/resource/ArchaeologicalProperty/0500590664
5 https://w3id.org/arco/resource/ArchaeologicalProperty/0500590665
6 https://w3id.org/arco/resource/ArchaeologicalProperty/0500590676
7 https://w3id.org/arco/resource/ArchaeologicalProperty/0500590677
8 https://w3id.org/arco/resource/ArchaeologicalProperty/0500590678
9 https://w3id.org/arco/resource/ArchaeologicalProperty/0500590679
10 https://w3id.org/arco/resource/ArchaeologicalProperty/0500590680
11 https://w3id.org/arco/resource/ArchaeologicalProperty/0500590681
12 https://w3id.org/arco/resource/ArchaeologicalProperty/0500590682
13 https://w3id.org/arco/resource/ArchaeologicalProperty/0500590689
14 https://w3id.org/arco/resource/ArchaeologicalProperty/0500590692
15 https://w3id.org/arco/resource/ArchaeologicalProperty/

In [17]:
museumThings = {}
for [(_,category)] in categories:
    a = run_query("""
    select distinct  ?site where {
        ?artwork a <""" + category + """> .
        ?artwork <https://w3id.org/arco/ontology/location/hasCulturalInstituteOrSite> ?culturalSite.
        
        ?culturalSite ?p ?site.
        ?site <http://dati.beniculturali.it/cis/siteAddress> ?address.
        ?address  <https://w3id.org/italia/onto/CLV/hasCity> <https://w3id.org/arco/resource/City/padova>.
        

    }
    LIMIT 6000
    """)
    print(category, len(a))
    museumThings[category] = a

Empty
https://w3id.org/arco/ontology/arco/TangibleCulturalProperty 0
https://w3id.org/arco/ontology/arco/MovableCulturalProperty 10
Empty
https://w3id.org/arco/ontology/arco/ImmovableCulturalProperty 0
https://w3id.org/arco/ontology/arco/ArchaeologicalProperty 3
Empty
https://w3id.org/arco/ontology/arco/ArchitecturalOrLandscapeHeritage 0
https://w3id.org/arco/ontology/arco/ScientificOrTechnologicalHeritage 2
https://w3id.org/arco/ontology/arco/PhotographicHeritage 1
https://w3id.org/arco/ontology/arco/NumismaticProperty 2
https://w3id.org/arco/ontology/arco/NaturalHeritage 2
Empty
https://w3id.org/arco/ontology/arco/MusicHeritage 0
https://w3id.org/arco/ontology/arco/HistoricOrArtisticProperty 4
Empty
https://w3id.org/arco/ontology/arco/ZoologicalHeritage 0
Empty
https://w3id.org/arco/ontology/arco/PetrologicHeritage 0
Empty
https://w3id.org/arco/ontology/arco/PalaeontologicalHeritage 0
Empty
https://w3id.org/arco/ontology/arco/MineralHeritage 0
Empty
https://w3id.org/arco/ontology/arc

In [18]:
museumProperties = {}

index = 0

for [(_,category)] in categories:
    for [(_,museum)] in museumThings[category]:
        a = run_query("""
        select distinct ?museum ?name ?img ?img2 ?desc ?type ?lat ?long where {

            BIND(<""" + museum + """> AS ?museum)
            OPTIONAL{?museum <http://www.w3.org/2000/01/rdf-schema#label> ?name.}
            OPTIONAL{?museum <http://xmlns.com/foaf/0.1/depiction> ?img. }
            OPTIONAL{?museum <http://data.cochrane.org/ontologies/pico/preview> ?img2. }
            OPTIONAL{?museum <https://w3id.org/arco/ontology/core/description> ?desc. }
            OPTIONAL{?museum <https://w3id.org/arco/ontology/location/hasSiteType> ?type. }

            OPTIONAL{
                ?museum <https://w3id.org/italia/onto/CLV/hasGeometry> ?geom.
                ?geom <https://w3id.org/arco/ontology/location/hasCoordinates> ?coord.
                ?coord <https://w3id.org/arco/ontology/location/lat> ?lat.
                ?coord <https://w3id.org/arco/ontology/location/long> ?long.
                
            }

        }
        LIMIT 1
        """)
        index = index + 1
        print(index, museum, a)
    
        museumProperties[museum] = a



1 http://dati.beniculturali.it/iccd/cf/resource/CulturalInstituteOrSite/1469799782221 [[('museum', 'http://dati.beniculturali.it/iccd/cf/resource/CulturalInstituteOrSite/1469799782221'), ('name', 'Palazzo Folco'), ('img', 'http://www.sigecweb.beniculturali.it/images/fullsize/ICCD1063415/ICCD14278510_padova_folco.jpg'), ('img2', 'http://www.sigecweb.beniculturali.it/images/fullsize/ICCD1063415/ICCD14278510_padova_folco.jpg'), ('desc', 'Nel 1939 il complesso – dopo il radicale intervento di restauro architettonico operato dalla Soprintendenza all’Arte Medievale e Moderna negli anni 1935-1939 su progetto dell’architetto Ferdinando Forlati, a seguito del riconoscimento della valenza architettonica ufficializzato con l’emanazione nel 1933 del vincolo di tutela – diventa sede della Regia Soprintendenza del Veneto, della Lombardia e della Venezia Tridentina, allora dipendente dalla Direzione Generale delle Antichità e Belle Arti del Ministero dell’Educazione nazionale'), ('type', 'https://w3i

In [19]:
import json
with open('artworkCategories.json', 'w') as outfile:
    json.dump(categories, outfile, indent=2)

with open('artworkList.json', 'w') as outfile:
    json.dump(things, outfile, indent=2)

with open('artworkProperties.json', 'w') as outfile:
    json.dump(properties, outfile, indent=2)

with open('siteList.json', 'w') as outfile:
    json.dump(museumThings, outfile, indent=2)

with open('siteProperties.json', 'w') as outfile:
    json.dump(museumProperties, outfile, indent=2)
